## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

In [ ]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Layer,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math

In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [ ]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from computeStats import computeOverAllDist, STAT_Descriptions

In [ ]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BBSBSBSB'
meas='PRCP'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

In [ ]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
Query="SELECT station, latitude,longitude,elevation,%s FROM weather"%feature
print(Query)
df1 = sqlContext.sql(Query)
df1.show(4)

In [ ]:
df2=df1.groupby(['station','latitude','longitude','elevation']).agg({"station": "count", feature: "mean"})
pdf=df2.toPandas()
pdf.sort_values(by=['station'],inplace=True)
pdf.head(10)

In [ ]:
pdf.shape

In [ ]:
#define a mapping from the range of the value to hex colors.
from matplotlib.colors import rgb2hex
_avg='avg(%s)'%feature
_min=pdf[_avg].min()
_max=pdf[_avg].max()
_min,_max

import pylab as plt
cmap=plt.get_cmap('jet')
def get_color(val):
    x=(val-_min)/(_max-_min)
    return(rgb2hex(cmap(x)[:3]))

get_color(1000.)

## Map

In [ ]:
# Get the minimum and maximum latitude and longitudes for our geographical area
geoArea = sqlContext.sql('select min(latitude), max(latitude), min(longitude), max(longitude) from weather').collect()[0]
min_lat = np.floor(geoArea[0])
max_lat = np.ceil(geoArea[1])
min_lon = np.floor(geoArea[2])
max_lon = np.ceil(geoArea[3])

box = (min_lat, max_lat, min_lon, max_lon)


In [ ]:
center = [(min_lat+max_lat)/2, (min_lon+max_lon)/2]
zoom = 9

#urlLayer = u'http://tile.stamen.com/terrain/{z}/{x}/{y}.jpg'
urlLayer = u'http://{s}.tile.opentopomap.org/{z}/{x}/{y}.png'
#urlLayer = u'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

m = Map(center=center, zoom=zoom, 
        default_tiles=TileLayer(url=urlLayer), scroll_wheel_zoom = True)

r = Rectangle(bounds=[[min_lat,min_lon],[max_lat,max_lon]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_lon-min_lon)/4
circles = []
for index,row in pdf.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    _count=row['count(station)']
    _coef=row[_avg]
    # taking sqrt of count so that the  area of the circle corresponds to the count
    c = Circle(location=(_lat,_long), radius=int(300*np.sqrt(_count+0.0)), weight=1,
            color='#F00', opacity=0.8, fill_opacity=0.4,
            fill_color=get_color(_coef))
    circles.append(c)
    m.add_layer(c)

m.add_layer(Marker(location=m.center, title='Center Label', opacity=0.8))

m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

# Create a topographical map with a legend

I am using folium to do this rather than ipyleaflet. Both use leaflet, but folium has better documentation and is more feature-filled.

In [ ]:
#!pip install folium
import folium
from folium import plugins, IFrame

In [ ]:
m = folium.Map(center, zoom_start=zoom, tiles='Stamen Terrain')

folium.features.RectangleMarker(
    bounds=[[min_lat,min_lon],[max_lat,max_lon]],
    color='blue',
    fill_color='red', fill_opacity=0.05,
    popup='Weather Service Data').add_to(m)


marker_cluster = folium.MarkerCluster().add_to(m)

for i in range(pdf.shape[0]):
    
    p = folium.Popup(IFrame("<b>Station: {}</b><br>Elevation: {}<br>Count={}<br>Average={:.2f}".\
                            format(pdf.ix[i,'station'], pdf.ix[i,'elevation'], \
                            pdf.ix[i,'count(station)'], pdf.ix[i,'avg(coeff_1)']), \
                            width=200, height=120))

    folium.Marker(
        location=[pdf.ix[i,'latitude'], pdf.ix[i,'longitude']],
        popup= p
    ).add_to(marker_cluster)

    
plugins.Fullscreen(
    position='topright',
    title='Enter fullscreen mode',
    titleCancel='Exit fullscreen mode dear Triton',
    forceSeparateButton=True).add_to(m)

m.save('mapOfRegion.html')
m

In [ ]:
m2 = folium.Map(center, zoom_start=zoom, tiles='Stamen Terrain')

folium.features.RectangleMarker(
    bounds=[[min_lat,min_lon],[max_lat,max_lon]],
    color='blue',
    fill_color='red', fill_opacity=0.05,
    popup='Weather Service Data').add_to(m2)


plugins.HeatMap(pdf[['latitude', 'longitude', 'avg(coeff_1)']].values).add_to(m2)
    
    
plugins.Fullscreen(
    position='topright',
    title='Enter fullscreen mode',
    titleCancel='Exit fullscreen mode dear Triton',
    forceSeparateButton=True).add_to(m2)

m2

In [ ]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

In [ ]:
RDD1=sc.parallelize(["spark  basics", "spark big  data analysis", "spring"]) 
RDD2=sc.parallelize(["spark using pyspark", "big data"])
 
RDD1.subtract(RDD2).collect()